<a href="https://colab.research.google.com/github/vpw/TSAI-END3/blob/main/Assignment11/Task3/Task3_BART_Para.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers

     |████████████████████████████████| 248 kB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 57.0 MB/s 
     |████████████████████████████████| 6.8 MB 49.0 MB/s 
     |████████████████████████████████| 1.7 MB 57.4 MB/s 
     |████████████████████████████████| 9.7 MB 32.3 MB/s 
     |████████████████████████████████| 43 kB 2.8 MB/s 
     |████████████████████████████████| 312 kB 61.4 MB/s 
     |████████████████████████████████| 1.2 MB 69.3 MB/s 
     |████████████████████████████████| 895 kB 64.2 MB/s 
     |████████████████████████████████| 596 kB 69.8 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 143 kB 58.4 MB/s 
     |████████████████████████████████| 97 kB 8.9 MB/s 
     |████████████████████████████████| 180 kB 56.0 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 133 kB 40.3 MB/s 
     |████████████████████████████████| 243 kB 51.1 MB/s 
     |█████████████████

In [ ]:
Data Preparation

We will be combining two datasets to serve as training data for our BART Paraphrasing Model.

1. Google PAWS-Wiki Labeled (Final)
2. Quora Question Pairs Dataset

In [2]:
!mkdir data
!wget https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz -P data
!tar -xvf data/paws_wiki_labeled_final.tar.gz -C data
!mv data/final/* data
!rm -r data/final

!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv -P data

--2022-01-29 06:39:08--  https://storage.googleapis.com/paws/english/paws_wiki_labeled_final.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.123.128, 142.250.98.128, 142.251.107.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.123.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4687157 (4.5M) [application/gzip]
Saving to: ‘data/paws_wiki_labeled_final.tar.gz’

paws_wiki_labeled_f 100%[===================>]   4.47M  --.-KB/s    in 0.05s   

2022-01-29 06:39:08 (94.0 MB/s) - ‘data/paws_wiki_labeled_final.tar.gz’ saved [4687157/4687157]

final/test.tsv
final/
final/train.tsv
final/dev.tsv
--2022-01-29 06:39:09--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [3]:
#from utils.py
import warnings

import pandas as pd

def load_data(
    file_path, input_text_column, target_text_column, label_column, keep_label=1
):
    df = pd.read_csv(file_path, sep="\t", error_bad_lines=False)
    df = df.loc[df[label_column] == keep_label]
    df = df.rename(
        columns={input_text_column: "input_text", target_text_column: "target_text"}
    )
    df = df[["input_text", "target_text"]]
    df["prefix"] = "paraphrase"

    return df


def clean_unnecessary_spaces(out_string):
    if not isinstance(out_string, str):
        warnings.warn(f">>> {out_string} <<< is not a string.")
        out_string = str(out_string)
    out_string = (
        out_string.replace(" .", ".")
        .replace(" ?", "?")
        .replace(" !", "!")
        .replace(" ,", ",")
        .replace(" ' ", "'")
        .replace(" n't", "n't")
        .replace(" 'm", "'m")
        .replace(" 's", "'s")
        .replace(" 've", "'ve")
        .replace(" 're", "'re")
    )
    return out_string

In [4]:
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

#from utils import load_data, clean_unnecessary_spaces

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

## Loading the two datasets 

In [34]:
# Google Data
train_df = pd.read_csv("data/train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("data/dev.tsv", sep="\t").astype(str)

train_df = train_df.loc[train_df["label"] == "1"]
eval_df = eval_df.loc[eval_df["label"] == "1"]

train_df = train_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)
eval_df = eval_df.rename(
    columns={"sentence1": "input_text", "sentence2": "target_text"}
)

train_df = train_df[["input_text", "target_text"]]
eval_df = eval_df[["input_text", "target_text"]]

train_df["prefix"] = "paraphrase"
eval_df["prefix"] = "paraphrase"

In [35]:
# take the first 10K and 5K from train and eval
train_df = train_df[:10000]
eval_df = eval_df[:5000]

In [36]:
print(train_df)

                                              input_text  ...      prefix
1      The NBA season of 1975 -- 76 was the 30th seas...  ...  paraphrase
3      When comparable rates of flow can be maintaine...  ...  paraphrase
4      It is the seat of Zerendi District in Akmola R...  ...  paraphrase
5      William Henry Henry Harman was born on 17 Febr...  ...  paraphrase
7      With a discrete amount of probabilities Formul...  ...  paraphrase
...                                                  ...  ...         ...
22584  The 8 talukas of this district are Devgad , Ka...  ...  paraphrase
22586  Neelankarai is located on the Old Mahabalipura...  ...  paraphrase
22588  In Singapore , ADCs who are officers of the Si...  ...  paraphrase
22589  In 1736 , Mackay 's widow sold the islands to ...  ...  paraphrase
22590  Justice Felix Reginald Dias Bandaranaike II ( ...  ...  paraphrase

[10000 rows x 3 columns]


In [37]:
print(eval_df)

                                             input_text  ...      prefix
1     They were there to enjoy us and they were ther...  ...  paraphrase
2     After the end of the war in June 1902 , Higgin...  ...  paraphrase
3     From the merger of the Four Rivers Council and...  ...  paraphrase
4     The group toured extensively and became famous...  ...  paraphrase
5     Kathy and her husband Pete Beale ( Peter Dean ...  ...  paraphrase
...                                                 ...  ...         ...
7993  In Advent , the traditional `` Tauberbischofsh...  ...  paraphrase
7994  In 2002 , the song was released by British pro...  ...  paraphrase
7995  Tommy Connolly , who plays Rory Jennings , pla...  ...  paraphrase
7996  Monroe Meadows , in Yosemite valley near Brida...  ...  paraphrase
7998  In 2014 the site launched iOS and Android appl...  ...  paraphrase

[3539 rows x 3 columns]


In [38]:
train_df.loc[[1,3,4]]

,input_text,target_text,prefix
1,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,paraphrase
3,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,paraphrase
4,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,paraphrase


In [39]:
eval_df.loc[[1,2,3]]

,input_text,target_text,prefix
1,They were there to enjoy us and they were ther...,They were there for us to enjoy and they were ...,paraphrase
2,"After the end of the war in June 1902 , Higgin...","In August , after the end of the war in June 1...",paraphrase
3,From the merger of the Four Rivers Council and...,Shawnee Trails Council was formed from the mer...,paraphrase


In [40]:
# Quora Data

# The Quora Dataset is not separated into train/test, so we do it manually the first time.
#df = load_data(
#    "data/quora_duplicate_questions.tsv", "question1", "question2", "is_duplicate"
#)
#q_train, q_test = train_test_split(df)

#q_train.to_csv("data/quora_train.tsv", sep="\t")
#q_test.to_csv("data/quora_test.tsv", sep="\t")

# The code block above only needs to be run once.
# After that, the two lines below are sufficient to load the Quora dataset.

q_train = pd.read_csv("data/quora_train.tsv", sep="\t")
q_test = pd.read_csv("data/quora_test.tsv", sep="\t")


In [41]:
# take 10K from train and 5K from test
q_train = q_train[:10000]
q_test = q_test[:5000]

In [42]:
print(q_train)

      Unnamed: 0  ...      prefix
0           3641  ...  paraphrase
1         326723  ...  paraphrase
2         169819  ...  paraphrase
3         324444  ...  paraphrase
4         279907  ...  paraphrase
...          ...  ...         ...
9995       69147  ...  paraphrase
9996      131252  ...  paraphrase
9997      329585  ...  paraphrase
9998       82725  ...  paraphrase
9999      212785  ...  paraphrase

[10000 rows x 4 columns]


In [43]:
print(q_test)

      Unnamed: 0  ...      prefix
0         269638  ...  paraphrase
1         402640  ...  paraphrase
2         202780  ...  paraphrase
3         304080  ...  paraphrase
4         198392  ...  paraphrase
...          ...  ...         ...
4995       84994  ...  paraphrase
4996      203232  ...  paraphrase
4997      272232  ...  paraphrase
4998      148143  ...  paraphrase
4999      338131  ...  paraphrase

[5000 rows x 4 columns]


In [44]:
train_df = pd.concat([train_df, q_train])
eval_df = pd.concat([eval_df, q_test])

train_df = train_df[["prefix", "input_text", "target_text"]]
eval_df = eval_df[["prefix", "input_text", "target_text"]]

train_df = train_df.dropna()
eval_df = eval_df.dropna()

train_df["input_text"] = train_df["input_text"].apply(clean_unnecessary_spaces)
train_df["target_text"] = train_df["target_text"].apply(clean_unnecessary_spaces)

eval_df["input_text"] = eval_df["input_text"].apply(clean_unnecessary_spaces)
eval_df["target_text"] = eval_df["target_text"].apply(clean_unnecessary_spaces)

In [45]:
print(train_df)

          prefix  ...                                        target_text
1     paraphrase  ...  The 1975 -- 76 season of the National Basketba...
3     paraphrase  ...  The results are high when comparable flow rate...
4     paraphrase  ...  It is the seat of the district of Zerendi in A...
5     paraphrase  ...  William Henry Harman was born in Waynesboro, V...
7     paraphrase  ...  Given a discrete set of probabilities formula ...
...          ...  ...                                                ...
9995  paraphrase  ...     How can we fix the current educational system?
9996  paraphrase  ...                 When will Spotify launch in India?
9997  paraphrase  ...              How can I drop porn and masturbation?
9998  paraphrase  ...                         What exactly is hard work?
9999  paraphrase  ...  How can I improve my english language skills? ...

[20000 rows x 3 columns]


In [46]:
print(eval_df)

          prefix  ...                                        target_text
1     paraphrase  ...  They were there for us to enjoy and they were ...
2     paraphrase  ...  In August, after the end of the war in June 19...
3     paraphrase  ...  Shawnee Trails Council was formed from the mer...
4     paraphrase  ...  The group toured extensively and was famous in...
5     paraphrase  ...  Kathy and her husband Peter Dean ( Pete Beale ...
...          ...  ...                                                ...
4995  paraphrase  ...  What would happen if everyone becomes vegetarian?
4996  paraphrase  ...  Is there anyway to get yourself to have a luci...
4997  paraphrase  ...    Which is the best question ever asked in Quora?
4998  paraphrase  ...  What are some of the best ways to gain weight ...
4999  paraphrase  ...  How do I self rehab from masturbation and porn...

[8539 rows x 3 columns]


In [47]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
#model_args.eval_batch_size = 64
model_args.eval_batch_size = 8
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
#model_args.fp16 = False
model_args.fp16 = True
model_args.learning_rate = 5e-5
#model_args.max_length = 128
model_args.max_length = 64
#model_args.max_seq_length = 128
model_args.max_seq_length = 64
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 8
model_args.use_multiprocessing = False
model_args.wandb_project = "Paraphrasing with BART"


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
)

model.train_model(train_df, eval_data=eval_df)

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)

# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)

with open(f"predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(eval_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "_____________________________________________________________________\n"
        )

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/971M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.9908107122483102}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/best_model
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/checkpoint-5000-epoch-2
INFO:simpletransformers.seq2seq.seq2seq_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/8539 [00:00<?, ?it/s]

INFO:simpletransformers.seq2seq.seq2seq_model:{'eval_loss': 0.9908107122483102}
INFO:simpletransformers.seq2seq.seq2seq_model:Saving model into outputs/
INFO:simpletransformers.seq2seq.seq2seq_model: Training of facebook/bart-large model complete. Saved to outputs/.


Generating outputs:   0%|          | 0/1068 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2343: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Some of the predictions from the preditions file are:

In 2009 he moved back to Philadelphia and lives in New York City today.

Truth:

He moved back to Philadelphia in 2009 and now lives in New York City.

Prediction:

In 2009 he moved back to Philadelphia and currently lives in New York City.

Paraphrase: In 2009 he moved back to Philadelphia and currently lives in New York City.

Paraphrase: In 2009 he moved back to Philadelphia and currently lives in New York City.

_____________________________________________________________________

Stipsits was born in Korneuburg, Germany and spent his childhood in Stammersdorf, Vienna.

Truth:

Stipsits was born in Korneuburg, and spent his childhood in Stammersdorf, Vienna.

Prediction:

Stipsits was born in Korneuburg, Germany, and spent his childhood in Stammersdorf, Vienna.

Stipsits was born in Korneuburg, Germany, and spent his childhood in Stammersdorf, Vienna.

Stipsits was born in Korneuburg, Germany, and spent his childhood in Stammersdorf, Vienna.

_____________________________________________________________________

Daudkhali is a village in Barisal Division in the Pirojpur district in southwestern Bangladesh.

Truth:

Daudkhali is a village in Pirojpur District in the Barisal Division of southwestern Bangladesh.

Prediction:

Daudkhali is a village in the Barisal Division in the Pirojpur District in southwestern Bangladesh.

Daudkhali is a village in the Barisal Division in the Pirojpur District in southwestern Bangladesh.

Daudkhali is a village in the Barisal Division in the Pirojpur District in southwestern Bangladesh.
_____________________________________________________________________

The temple serves as the cultural and religious center for Korean Hindus and immigrants from South Asian countries.

Truth:

The temple serves as a cultural and religious centre for Korean Hindus and immigrants from the South Asian countries.

Prediction:

Paraphrase : The temple serves as the cultural and religious centre for Korean Hindus and immigrants from South Asian countries.

Paraphrase : The temple serves as the cultural and religious centre for Korean Hindus and immigrants from South Asian countries.

Paraphrase : The temple serves as the cultural and religious centre for Korean Hindus and immigrants from South Asian countries.
_____________________________________________________________________

When a surface has a constant zero developable curvature, then it is a Euclidean surface and the geometry of the surface is Gaussian geometry.

Truth:

If a surface has a constant developable curvature of zero, then it is an euclidean surface and the geometry of the surface is a Gaussian geometry.

Prediction:

Paraphrase : When a surface has a constant zero developable curvature, then it is a Euclidean surface and the geometry of the surface is Gaussian geometry.

Paraphrase : When a surface has a constant zero developable curvature, then it is a Euclidean surface and the geometry of the surface is Gaussian geometry.

Paraphrase : When a surface has a constant zero developable curvature, then it is a Euclidean surface and the geometry of the surface is Gaussian geometry.
